In [ ]:
import sys
sys.path.insert(0,r'/Users/Shared/c/CodeRepository/Formatting-Error-Correction/')
import ast
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import seaborn as sns
from utils.System_Evaluation_Metrics import precision_at_k, mrr
sns.set_theme()
sns.set_style("whitegrid",{'axes.grid' : False})

In [ ]:
def show_values_on_bars(axs, h_v="v", space=0.4):
    def _show_on_single_plot(ax):
        if h_v == "v":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() / 2
                _y = p.get_y() + p.get_height()
                value = int(p.get_height())
                ax.text(_x, _y, value, ha="center") 
        elif h_v == "h":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() + float(space)
                _y = p.get_y() + p.get_height()
                value = int(p.get_width())
                ax.text(_x, _y, value, ha="left")

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _show_on_single_plot(ax)
    else:
        _show_on_single_plot(axs)

In [ ]:
print('Enter name of excel file:')
nameExcel = input()

In [ ]:
errorFixingStats = pd.read_excel(nameExcel + '.xlsx')
display(errorFixingStats)

#### 1. Kind of Error

In [ ]:
kindOfError = dict(errorFixingStats['Kind of Fix ( D, A ,R)'].value_counts())
kindOfError
plt.figure(figsize = [8,6])
plt.bar(kindOfError.keys(), kindOfError.values(), width = 0.4)
plt.title('Type of Error per #Appearances')
plt.xlabel('Type of Error')
plt.xticks(list(kindOfError.keys()), ['Delete','Append','Replace'])
plt.yticks([0,10,20,30,40,50,60,70,80,90])
plt.ylabel('Number of Appearances');

In [ ]:
kindOfError

In [ ]:
fix = dict(errorFixingStats['Fix/No Fix'].value_counts())
print(fix)
plt.figure(figsize = [8,6])
plt.bar(fix.keys(), fix.values(), width = 0.65)
plt.title('Fix/No Fix')
plt.xlabel('Answer')
plt.xticks(list(fix.keys()), ['Yes','No'])
plt.yticks([0,10,20,30,40,50,60,70,80,90])
plt.ylabel('Number of Appearances');

#### 2. Best Score

In [ ]:
numOfBestScore = dict(errorFixingStats['Does the correct fixed file has the best score?'].value_counts())
numOfBestScore

plt.figure(figsize = [8,6])
plt.bar(numOfBestScore.keys(), numOfBestScore.values(), width = 0.4)
plt.title('Does the fixed file have the best formation score across the suggested fixes?')
plt.xlabel('Type of Error')
plt.xticks(list(numOfBestScore.keys()), ['Yes','No','No Fix'])
plt.yticks([0,10,20,30,40,50,60,70,80,90])
plt.ylabel('Number of Appearances');

In [ ]:
numOfSugg = dict(errorFixingStats['Total number of suggested fixed files'].value_counts())
print(numOfSugg)
plt.figure(figsize = [8,6])
plt.bar(numOfSugg.keys(), numOfSugg.values(), width = 0.6)
plt.title('#Suggested Fixes')
plt.xlabel('Number of Suggested Fixes')
plt.ylabel('Number of Appearances')
plt.xticks(list(numOfSugg.keys()))
plt.yticks([0,10,20,30,40,50,60,70,80,90]);


In [ ]:
rankOfFix = dict(errorFixingStats['Rank of correct fixed file along fixes (according to score)'].value_counts())
print(rankOfFix)
plt.figure(figsize = [8,6])
plt.bar(rankOfFix.keys(), rankOfFix.values(), width = 0.4)
plt.title('Rank of Correct Fix')
plt.xlabel('Rank')
plt.xticks(list(rankOfFix.keys()), ['1','2','No Fix'])
plt.yticks([0,10,20,30,40,50,60,70,80,90])
plt.ylabel('Number of Appearances');

In [ ]:
scoreReduction = 100*errorFixingStats[errorFixingStats['Score Reduction ( %)'] != 'No Reduction']['Score Reduction ( %)']
fig = plt.figure(figsize = [8,6])
ax = sns.histplot(x = scoreReduction, bins = [0,5,10,15,20,25,30,35,40,45,50,55,60],kde = True)
show_values_on_bars(ax,"v",0.1)
plt.title('Score Reduction Distribution')
plt.ylabel('')
plt.xlabel('Score Reduction (%)')
plt.xticks([0,5,10,15,20,25,30,35,40,45,50,55,60])
plt.yticks([0,5,10,15,20]);


In [ ]:
lowerScores = pd.read_csv('LowerScores.csv', names=['Scores'])
initialScores = pd.read_csv('InitialScores.csv', names=['InitialScore'])
suggFixes = errorFixingStats['Total number of suggested fixed files']
fileName = errorFixingStats['File Name']

In [ ]:
fixScoresPerFile = {}
initScorePerFile = {}
i = 0
for num,file in zip(suggFixes,fileName):
    fixScoresPerFile.update({file:list(lowerScores['Scores'][i:i+num])})
    i += num

for initScore,file in zip(initialScores['InitialScore'],fileName):
    initScorePerFile.update({file:initScore})

In [ ]:
scoreReduction_noCorrFix = []
for fileName in initScorePerFile.keys():
    initScore = initScorePerFile.get(fileName)
    lowerScores = fixScoresPerFile.get(fileName)
    reduction = [100*(initScore - lowerScore)/initScore for lowerScore in lowerScores if lowerScore != min(lowerScores)] 
    scoreReduction_noCorrFix.extend(reduction)

In [ ]:
fig = plt.figure(figsize = [8,6])
ax = sns.histplot(x = scoreReduction_noCorrFix, bins = [0,5,10,15,20,25,30,35,40,45])
show_values_on_bars(ax,"v",0.1)
plt.title('1. Score Reduction Distribution (Suggestions - No Correct Fix)')
plt.ylabel('')
plt.xlabel('Score Reduction (%)')
plt.xticks([0,5,10,15,20,25,30,35,40,45]);